In [1]:
#Data Analysis
import numpy as np
import os
import json
trainexamples = []
testexamples = []

def loadtest(direc):
    for each in os.listdir(direc):        
        if '.npy' in each:
            fids = each.split('_')[1][:-4]
            examples = np.load(direc+'/'+each)
            with codecs.open(direc+'/'+each[:-4]+'.json', 'r','utf-8') as infile:
                values = json.load(infile)
            assert(len(examples) == len(values))
            yield examples,fids,values
    return

   
def load(direc):
    examples = []
    fids  = []
    values = []
    for each in os.listdir(direc):        
        try:
            if '.npy' in each:
                fids.append(each.split('_')[1][:-4])
                examples.extend(np.load(direc+'/'+each))
                with open(direc+'/'+each[:-4]+'.json', 'r') as infile:
                    values.extend(json.load(infile))
        except:
            continue
    return examples,fids,values

trainexamples, trainfids, trainvalues = load('../../../data/Titlefeature/train')

trainlabels, trainfeatures = [], []
for each in trainexamples:
    trainfeatures.append(each[1:])
    trainlabels.append(each[0])

assert(len(trainvalues)==len(trainlabels))
print len(trainfeatures[0])
print len(trainfeatures)

X = np.array(trainfeatures)
print X.shape
Y = map(lambda x: 1 if x==1 else 0, trainlabels )
Y = np.array(Y)
print 'No of positive examples %d' %Y.nonzero()[0].shape[0]
print 'Total number of examples %d'%Y.shape[0]

2
12956
(12956, 2)
No of positive examples 50
Total number of examples 12956


In [4]:
import numpy as np


with open('../../../data/Titlefeature/train/featurenames.json') as infile:
    featurenames = json.load(infile)
assert(len(featurenames) == len(trainfeatures[0]))


################ ML methods ######################################


import random

from time import time
from sklearn.svm import SVC
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.grid_search import RandomizedSearchCV
from operator import itemgetter
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint as sp_randint
from sklearn.cross_validation import StratifiedKFold



d = dict(zip(featurenames,range(len(featurenames))))
print '-----------------'*10
print 'FeatureNames'
for each in d.keys():
    print each
print '-----------------'*10
###################################################################################################################

# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        if i == 0:
            bestparams = (score.parameters,score.mean_validation_score)
        print("")
    return bestparams

def getf1score(estimator, X, Y):
    #return recall_score(estimator.predict(X),Y)
    return f1_score(estimator.predict(X),Y)



clf1 = SVC(C=1.0)
clf2 = LogisticRegression(penalty='l2', random_state=4,C=1.0)
clf3 = RandomForestClassifier(n_estimators=20)
s3fold = StratifiedKFold(y=Y, n_folds=5,
                                         shuffle=True, random_state=2)
bestparams = []

C = [0.001, 0.001, 0.1, 0.7, 0.5, 0.6, 0.9, 1, 2, 5, 7, 10,50, 70, 100,1000]
gamma = [0.00001, 0.0001,0.001, 0.01, 0.1]
class_weights = [{0:1,1:1},{0:1,1:5},{0:1,1:10},{0:1,1:15},{0:1,1:20}]

param_dist = {"C": C, "gamma":gamma, "class_weight":class_weights}

Xnew = X


print '#################### SVC  ###############################'
np.random.seed(5)
# # run randomized search
n_iter_search = 20
# random_search = RandomizedSearchCV(clf1, param_distributions=param_dist, scoring=getf1score,
#                                    verbose=2, cv=s3fold )
# start = time()
# random_search.fit(Xnew, Y)
# print("RandomizedSearchCV took %.2f seconds for %d candidates"
#       " parameter settings." % ((time() - start), n_iter_search))
# bestparams.append(report(random_search.grid_scores_))
# print 'Continue ??'
# raw_input()

param_dist = {"C": C,"class_weight": class_weights}
print '#################### LOGISTIC REGRESSION ###############################'
random_search = RandomizedSearchCV(clf2, param_distributions=param_dist, scoring=getf1score,
                                   verbose=2 ,cv=s3fold)

random_search.fit(Xnew, Y)
start = time()
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
bestparams.append(report(random_search.grid_scores_))
print 'Continue ??'
raw_input()



# specify parameters and distributions to sample from
print '#################### RANDOM FOREST ###############################'
param_dist = {"max_depth": [3, None],
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]
             }
random_search = RandomizedSearchCV(clf3, param_distributions=param_dist, scoring=getf1score,
                                   verbose=2 )


start = time()
random_search.fit(Xnew, Y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
bestparams.append(report(random_search.grid_scores_))
print 'BestParams and score'
print bestparams



--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
FeatureNames
isbestTitle
isheading
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#################### SVC  ###############################
#################### LOGISTIC REGRESSION ###############################
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] C=1000, class_weight={0: 1, 1: 15} ..............................
[CV] ..................... C=1000, class_weight={0: 1, 1: 15} -   0.0s
[CV] C=1000, class_weight={0: 1, 1: 15} ..............................
[CV] ..................... C=1000, class_weight={0: 1, 1: 15} -   0.0s
[CV] C=1000, class_weight={0: 1, 1: 15} ..............................
[CV] ..................... C=1000, class_weight={0: 1, 1: 15} -   0.0s
[CV] 

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s


[CV] ...................... C=100, class_weight={0: 1, 1: 15} -   0.0s
[CV] C=100, class_weight={0: 1, 1: 15} ...............................
[CV] ...................... C=100, class_weight={0: 1, 1: 15} -   0.0s
[CV] C=100, class_weight={0: 1, 1: 15} ...............................
[CV] ...................... C=100, class_weight={0: 1, 1: 15} -   0.0s
[CV] C=100, class_weight={0: 1, 1: 15} ...............................
[CV] ...................... C=100, class_weight={0: 1, 1: 15} -   0.0s
[CV] C=0.001, class_weight={0: 1, 1: 15} .............................
[CV] .................... C=0.001, class_weight={0: 1, 1: 15} -   0.0s
[CV] C=0.001, class_weight={0: 1, 1: 15} .............................
[CV] .................... C=0.001, class_weight={0: 1, 1: 15} -   0.0s
[CV] C=0.001, class_weight={0: 1, 1: 15} .............................
[CV] .................... C=0.001, class_weight={0: 1, 1: 15} -   0.0s
[CV] C=0.001, class_weight={0: 1, 1: 15} .............................
[CV] .

[Parallel(n_jobs=1)]: Done  41 jobs       | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.6s finished


[CV] ....................... C=0.9, class_weight={0: 1, 1: 1} -   0.0s
[CV] C=0.9, class_weight={0: 1, 1: 1} ................................
[CV] ....................... C=0.9, class_weight={0: 1, 1: 1} -   0.0s
[CV] C=0.9, class_weight={0: 1, 1: 1} ................................
[CV] ....................... C=0.9, class_weight={0: 1, 1: 1} -   0.0s
[CV] C=0.9, class_weight={0: 1, 1: 1} ................................
[CV] ....................... C=0.9, class_weight={0: 1, 1: 1} -   0.0s
[CV] C=0.9, class_weight={0: 1, 1: 1} ................................
[CV] ....................... C=0.9, class_weight={0: 1, 1: 1} -   0.0s
[CV] C=0.001, class_weight={0: 1, 1: 10} .............................
[CV] .................... C=0.001, class_weight={0: 1, 1: 10} -   0.0s
[CV] C=0.001, class_weight={0: 1, 1: 10} .............................
[CV] .................... C=0.001, class_weight={0: 1, 1: 10} -   0.0s
[CV] C=0.001, class_weight={0: 1, 1: 10} .............................
[CV] .

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s


[CV]  min_samples_split=1, bootstrap=False, criterion=entropy, max_depth=3, min_samples_leaf=5 -   0.0s
[CV] min_samples_split=1, bootstrap=False, criterion=entropy, max_depth=3, min_samples_leaf=5 
[CV]  min_samples_split=1, bootstrap=False, criterion=entropy, max_depth=3, min_samples_leaf=5 -   0.0s
[CV] min_samples_split=6, bootstrap=True, criterion=entropy, max_depth=3, min_samples_leaf=10 
[CV]  min_samples_split=6, bootstrap=True, criterion=entropy, max_depth=3, min_samples_leaf=10 -   0.0s
[CV] min_samples_split=6, bootstrap=True, criterion=entropy, max_depth=3, min_samples_leaf=10 
[CV]  min_samples_split=6, bootstrap=True, criterion=entropy, max_depth=3, min_samples_leaf=10 -   0.0s
[CV] min_samples_split=6, bootstrap=True, criterion=entropy, max_depth=3, min_samples_leaf=10 
[CV]  min_samples_split=6, bootstrap=True, criterion=entropy, max_depth=3, min_samples_leaf=10 -   0.0s
[CV] min_samples_split=5, bootstrap=True, criterion=gini, max_depth=3, min_samples_leaf=6 
[CV]  min

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.8s finished


In [7]:
import pickle
from sklearn.metrics import recall_score
random.seed(124)  
np.random.seed(124)
# clf = RandomForestClassifier(n_estimators=20, bootstrap=True, min_samples_leaf=1, 
#                              min_samples_split=1, criterion='gini', max_features=5, max_depth=None)
clf = LogisticRegression(**{'C': 1000, 'class_weight': {0: 1, 1: 15}})

rpos = filter(lambda x: Y[x]>0, range(Y.shape[0]))
rneg = filter(lambda x: Y[x]==0, range(Y.shape[0]))

random.shuffle(rpos)
random.shuffle(rneg)

rtrain = rpos[:int(len(rpos)/2)]+rneg[:int(len(rneg)/3)]
rtest = rpos[int(len(rpos)/2):]+rneg[int(len(rneg)/3):]
random.shuffle(rtrain)
random.shuffle(rtest)


clf.fit(X[rtrain,:],Y[rtrain])
print f1_score(clf.predict(X[rtest,:]),Y[rtest])
print recall_score(clf.predict(X[rtest,:]),Y[rtest])
print len(clf.predict(X[rtest,:]).nonzero()[0]),len(Y[rtest].nonzero()[0])
print 


print 'Pickling...'
clf.fit(X,Y)
print clf.coef_, clf.intercept_
pickle.dump(clf, file('../../../data/Titlefeature/train/Title.classifier','w'))
print 'Done pickling..'



0.758620689655
0.666666666667
33 25

Pickling...
[[ 8.06406635  3.72745336]] [-5.92377101]
Done pickling..


In [ ]:
class TitleClassifier:
    
    def __init__(self):
        """Predicts for a single news article only"""
        pass
    
    def predict(self,X):        
        assert(X.shape[1] == 2)        
        rowno = 0
        res = -1
        gotit = False
        for goodmatch,isheading in X:
            
            if goodmatch == 1 and isheading==1:
                res = rowno
                
                break;
            if isheading == 1 and not gotit:
                res = rowno
                gotit  = True
            rowno+=1
        Y = [0]*X.shape[0]
        print res
        if res>=0:
            Y[res] = 1
        return Y
            
TestCheck = TitleClassifier()
import pickle
pickle.dump( TestCheck, file('Best_title.dat','wb'))
TC = pickle.load(file('Best_title.dat','rb'))
for x,y,v in loadtest('../../../../data/Titlefeature/cv1/test'):
    Y = TC.predict(np.array(x).transpose()[1:,].transpose())
    print '--**'*10
    title = ''
    for i,e in enumerate(Y):
        if e == 1:
            title+=v[i]
    print title
     